### The Scenario

You are an analyst for an e-commerce company. The marketing team wants to launch a new loyalty program and needs your help identifying high-value customers based on their purchasing habits.

### Database Schema and Sample Data

First, here is the schema and sample data for our four tables: `customers`, `products`, `orders`, and `order_items`. You can run this in any standard SQL environment (like PostgreSQL, MySQL, or an online SQL fiddle) to practice.

```sql
-- Create tables
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    name VARCHAR(50),
    join_date DATE
);

CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(50),
    category VARCHAR(50),
    price DECIMAL(10, 2)
);

CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE order_items (
    order_id INT,
    product_id INT,
    quantity INT,
    PRIMARY KEY (order_id, product_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

-- Insert sample data
INSERT INTO customers VALUES
(1, 'Alice', '2024-01-15'),
(2, 'Bob', '2024-02-20'),
(3, 'Charlie', '2023-11-05');

INSERT INTO products VALUES
(101, 'Laptop', 'Electronics', 1200.00),
(102, 'Mouse', 'Electronics', 25.00),
(201, 'Coffee Maker', 'Home Goods', 80.00),
(202, 'Blender', 'Home Goods', 45.00),
(301, 'SQL Book', 'Books', 55.00);

INSERT INTO orders VALUES
(1001, 1, '2024-03-10'),
(1002, 2, '2024-03-12'),
(1003, 1, '2024-04-02'),
(1004, 3, '2024-04-05'),
(1005, 2, '2025-01-20'),
(1006, 1, '2025-02-15');

INSERT INTO order_items VALUES
(1001, 101, 1),
(1001, 102, 1),
(1002, 201, 1),
(1002, 301, 2),
(1003, 102, 3),
(1004, 202, 1),
(1004, 201, 1),
(1005, 101, 1),
(1006, 301, 1);

```

-----

### The Challenge: A Multi-Part Quest

You need to answer a series of questions that build on each other.

#### Part 1: Customer Total Spending

**Goal:** Write a query to calculate the total amount of money each customer has spent. The result should show the customer's name and their total spending, ordered from highest to lowest.

**Skills Tested:**

  * `INNER JOIN` across multiple tables.
  * `GROUP BY` for aggregation.
  * `SUM()` aggregate function.
  * `ORDER BY`.

#### Part 2: Recent High-Value Customers

**Goal:** Modify the previous query to identify high-value customers based on their spending **in the year 2024 only**.

**Skills Tested:**

  * `WHERE` clause with date filtering.
  * Combining filtering with joins and aggregations.

#### Part 3: Top Spender in Each Category (The Hard Part)

**Goal:** This is the main challenge. Write a query to find the **single top-spending customer for each product category**. The final output should show the category, the customer's name, and the total amount they spent in that category.

**Skills Tested:**

  * **Common Table Expressions (CTEs)** to build a logical, multi-step query.
  * **Window Functions** (`RANK()` or `ROW_NUMBER()`) to rank customers within groups.
  * `PARTITION BY` to create windows for ranking.

-----

### Solution and Explanation

Here are the solutions. It's best to try solving them yourself before looking\!

#### Solution to Part 1

```sql
SELECT
    c.name,
    SUM(p.price * oi.quantity) AS total_spending
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY c.name
ORDER BY total_spending DESC;
```

#### Solution to Part 2

```sql
SELECT
    c.name,
    SUM(p.price * oi.quantity) AS total_spending_2024
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE strftime('%Y', o.order_date) = '2024' -- Use YEAR(o.order_date) in MySQL
GROUP BY c.name
ORDER BY total_spending_2024 DESC;
```

#### Solution to Part 3

This solution uses a CTE and a window function, which is a clean and powerful way to solve this common type of "ranking within a group" problem.

```sql
-- Use a Common Table Expression (CTE) to first calculate spending per customer per category
WITH CustomerCategorySpending AS (
    SELECT
        p.category,
        c.name AS customer_name,
        SUM(p.price * oi.quantity) AS category_spending,
        -- Now, rank customers within each category based on their spending
        RANK() OVER(PARTITION BY p.category ORDER BY SUM(p.price * oi.quantity) DESC) as spending_rank
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    GROUP BY
        p.category,
        c.name
)
-- Finally, select only the top-ranked customer from each category
SELECT
    category,
    customer_name,
    category_spending
FROM CustomerCategorySpending
WHERE spending_rank = 1;

```

**Explanation:**

1.  **`CustomerCategorySpending` CTE**: We first create a temporary, named result set that calculates how much each customer spent in each category.
2.  **`RANK() OVER(...)`**: This is the window function.
      * `PARTITION BY p.category`: It tells the function to perform the ranking separately for each category ('Electronics', 'Home Goods', etc.).
      * `ORDER BY SUM(...) DESC`: Within each partition, it ranks customers based on their spending in descending order.
3.  **Final `SELECT`**: The outer query simply selects from the CTE, filtering for only the rows where `spending_rank` is 1.